In [249]:
import sys; sys.path.append("..")

In [250]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from data_processing.data_generate import generate_random_matrix


## 1.Factorization matrix

$$Loss = min \sum_{q^*,p^* (u,i) \in K} (R_ui - q{_i}^{T} p_u)^2 + \lambda(||q_i||^2 + ||p_u||^2)\\$$

Find the partial derivative of qi: $$ - 2(r_{ui} - q_i^Tp_u)p_u + 2 \lambda q_i$$

Find the partial derivative of pu $$- 2(r_{ui} - q_i^Tp_u)q_i + 2 \lambda p_u$$


Then Update parameters $$q_i = q_i - n\cdot(- 2(r_{ui} - q_i^Tp_u)p_u + 2 \lambda q_i)$$

$$p_u = p_u - n\cdot (-2(r_{ui} - q_i^Tp_u)q_i + 2 \lambda p_u )$$

In [251]:
#使用for循环方法
def matrix_factorization_original(Matrix, k, learning_rate=0.01, reg_param=0.1, iterations=1000, seed=None):
    """
    Matrix: 评分矩阵
    k: 隐特征维度
    learning_rate: 学习率
    iterations: 迭代次数
    seed: 随机数种子
    """
    
    if seed is not None:
        np.random.seed(seed)

    rows, cols = Matrix.shape

    # Initialize user matrix and item matrix
    user_matrix = np.random.rand(rows, k).astype(np.float32)
    item_matrix = np.random.rand(k, cols).astype(np.float32)
    
    pbar = tqdm(range(iterations), desc="Training Progress", ncols=100, unit="iter")

    for iter in pbar:
        total_loss = 0  # 用来计算每次的总误差
        reg_loss = 0  # 用来计算每次的正则化损失

        for i in range(rows):
            for j in range(cols):
                if Matrix[i, j] > 0:
                    error = Matrix[i, j] - np.dot(item_matrix[:, j], user_matrix[i, :])

                    # 计算物品矩阵的梯度
                    q_i = item_matrix[:, j].copy()
                    q_grad = -2 * error * user_matrix[i, :] + 2 * reg_param * item_matrix[:, j]
                    item_matrix[:, j] -= learning_rate * q_grad

                    # 计算用户矩阵的梯度
                    p_grad = -2 * error * q_i + 2 * reg_param * user_matrix[i, :]
                    user_matrix[i, :] -= learning_rate * p_grad

                    # 计算总损失
                    total_loss += (error ** 2)
                    
        pbar.set_postfix(error=total_loss)
         
    return user_matrix, item_matrix


In [252]:
#构建矩阵
matrix = generate_random_matrix(rows = 8,cols = 8,min_value = 0, max_value = 5,seed = 42)
matrix

array([[3, 4, 2, 4, 4, 1, 2, 2],
       [2, 4, 3, 2, 5, 4, 1, 3],
       [5, 5, 1, 3, 4, 0, 3, 1],
       [5, 4, 3, 0, 0, 2, 2, 1],
       [3, 3, 5, 5, 5, 2, 3, 3],
       [0, 2, 4, 2, 4, 0, 1, 3],
       [0, 3, 5, 1, 1, 0, 1, 4],
       [1, 3, 3, 3, 3, 4, 2, 5]])

In [253]:
#计算隐向量
user_matrix, item_matrix = matrix_factorization_original(matrix, k = 5, iterations = 1000, reg_param = 0.01,seed = 42)

Training Progress: 100%|████████████████████████| 1000/1000 [00:01<00:00, 631.78iter/s, error=0.776]


In [254]:
res1 = pd.DataFrame(np.dot(user_matrix ,item_matrix))
res1

,0,1,2,3,4,5,6,7
0,3.038984,3.735725,1.927041,3.649276,4.083481,1.045395,2.479229,2.041741
1,2.012510,3.932389,2.994193,1.937654,4.979774,3.970447,1.118679,3.017461
2,4.960433,5.062543,1.044745,3.166369,3.939931,0.515362,2.743860,0.968924
3,4.956802,3.997909,3.000133,1.903604,2.559543,1.975332,1.985110,1.019623
4,2.989860,3.043139,4.973872,4.994226,4.970627,1.998876,2.923547,3.003056
5,0.858340,2.053362,3.984272,2.095200,3.946965,3.579638,0.893598,2.973585
6,2.895861,2.929444,4.944027,0.946600,1.040376,5.475634,1.104696,3.994296
7,0.994989,3.064798,3.040766,3.069566,2.980732,3.972241,1.847939,4.939415


In [255]:
#original matrix
pd.DataFrame(matrix)

,0,1,2,3,4,5,6,7
0,3,4,2,4,4,1,2,2
1,2,4,3,2,5,4,1,3
2,5,5,1,3,4,0,3,1
3,5,4,3,0,0,2,2,1
4,3,3,5,5,5,2,3,3
5,0,2,4,2,4,0,1,3
6,0,3,5,1,1,0,1,4
7,1,3,3,3,3,4,2,5


## 1.1summary
通过矩阵分解,我们就可以成功的预测用户对未评价的物品的评分的预测值,如对于第6行第7列(index[5,6])的物品,矩阵的预测值3.86\
\
Through matrix decomposition, we can successfully predict the predicted value of the user's rating for unevaluated items. For example, for the item in row 6 and column 7 (index[5,6]), the predicted value of the matrix is ​​3.86

## 2 利用矩阵计算

$$E = (R - \hat{R})$$

更新用户矩阵 (U)：$$U = U + \eta \cdot \left( 2 \cdot E \cdot V^T - 2\lambda U \right)$$

更新物品矩阵 (V)：$$V = V + \eta \cdot \left( 2 \cdot E^T \cdot U - 2\lambda V \right)$$

其中
- $(\hat{R} \in \mathbb{R}^{m \times n})$: 预测评分矩阵。
- $(U \in \mathbb{R}^{m \times k}): $用户矩阵，每一行表示一个用户的特征向量 $(p_u)$。
- $(V \in \mathbb{R}^{k \times n}$: 物品矩阵，每一列表示一个物品的特征向量 $(q_i)$。

In [291]:
def matrix_factorization(matrix,k, learning_rate = 0.01, reg_param=0.1,iterations = 1000, seed = 42):
    """
    matrix: 评分矩阵
    k: 隐特征维度
    learning_rate: 学习率
    iterations: 迭代次数
    seed: 随机数种子
    """
    
    if seed:
        np.random.seed(seed)

    rows,cols = matrix.shape

    # Initialize user matrix and item matrix
    user_matrix = np.random.rand(rows, k)
    item_matrix = np.random.rand(k,cols)

    #Only use non-zero elements in the matrix to optimize
    mask = (matrix > 0).astype(np.float32) 
    
    pbar = tqdm(range(iterations),desc= "Training",ncols = 100, unit = 'iter')
    for i in pbar:
        Predicted = np.dot(user_matrix,item_matrix)
        #这里值计算了非零元素的损失
        error = (matrix - Predicted) * mask

        #Gradient calculation and update

        user_grad = -2 *np.dot(error,item_matrix.T) + 2 * reg_param * user_matrix
        user_matrix -= learning_rate * (user_grad)

        item_grad = -2 * np.dot(user_matrix.T, error) + 2 * reg_param * item_matrix
        item_matrix -= learning_rate * item_grad

        #Calculate the error
        mse = np.sum(error ** 2) 
        pbar.set_postfix(MSE=mse)  # 显示MSE和正则化损失
    

    return user_matrix,item_matrix



In [292]:
#构建矩阵
matrix = generate_random_matrix(rows = 8,cols = 8,min_value = 0, max_value = 5,seed = 42)
matrix

array([[3, 4, 2, 4, 4, 1, 2, 2],
       [2, 4, 3, 2, 5, 4, 1, 3],
       [5, 5, 1, 3, 4, 0, 3, 1],
       [5, 4, 3, 0, 0, 2, 2, 1],
       [3, 3, 5, 5, 5, 2, 3, 3],
       [0, 2, 4, 2, 4, 0, 1, 3],
       [0, 3, 5, 1, 1, 0, 1, 4],
       [1, 3, 3, 3, 3, 4, 2, 5]])

In [293]:
user_matrix, item_matrix = matrix_factorization(matrix, k = 5, iterations = 1000, reg_param = 0.01)

Training: 100%|██████████████████████████████████| 1000/1000 [00:00<00:00, 1177.78iter/s, MSE=0.623]


In [286]:
res = pd.DataFrame(np.dot(user_matrix ,item_matrix))
res

,0,1,2,3,4,5,6,7
0,3.019627,3.809476,1.962692,3.681781,4.136928,1.013876,2.475588,2.056890
1,2.000868,3.973364,2.998601,1.951341,5.013391,3.994376,1.075187,3.009171
2,4.984229,5.104251,1.021969,3.190802,3.915346,0.463083,2.714706,0.961644
3,4.995728,4.000230,3.000138,2.170694,2.978890,1.993152,1.991280,1.007326
4,2.994165,3.033363,5.001991,5.040917,4.975729,1.998797,2.926847,2.990339
5,0.850091,2.048258,4.001711,2.096305,3.956830,3.662068,0.861832,2.981186
6,3.426032,2.961785,4.987558,0.940668,1.030728,5.900869,1.096164,4.006254
7,0.997361,3.051292,3.013232,3.084434,2.962409,3.991305,1.861287,4.979314


In [287]:
#original matrix
pd.DataFrame(matrix)

,0,1,2,3,4,5,6,7
0,3,4,2,4,4,1,2,2
1,2,4,3,2,5,4,1,3
2,5,5,1,3,4,0,3,1
3,5,4,3,0,0,2,2,1
4,3,3,5,5,5,2,3,3
5,0,2,4,2,4,0,1,3
6,0,3,5,1,1,0,1,4
7,1,3,3,3,3,4,2,5


### 3 消除用户和物品打分的偏差

由于不同用户的打分体系不同（比如在5分为满分的情况下，有的用户认为打3分已经是很低的分数了，而有的用户认为打1分才是比较差的评价），不同物品的衡量标准也有所区别（比如电子产品的平均分和日用品的平均分差异有可能比较大），**为了消除用户和物品打分的偏差(Bias)，常用的做法是在矩阵分解时加入用户和物品的偏差向量，如（式2-10）所示。**

$$r_ui = u + b_i +b_u + q_i^Tp_u$$

$$L = \sum_{(u,i) \in \text{non-zero entries}} \left[ (r_{ui} - (u + b_u + b_i + p_u^T q_i))^2 \right] + \lambda \left(||p_u||^2 + ||q_i||^2 + b_u^2 + b_i^2 \right)
$$

其中
- $r_ui$ 是用户 `u` 对物品 `i` 的评分。  
- $μ$ 是全局偏差，$b_u$ 是用户偏差，$`b_i`$ 是物品偏差。  b_i 可以使用使用物品i收到的所有评分的均值，**$b_u$是用户偏差系数，可以使用用户u给出所有评分的均值**
- $p_u$ 和 $q_i$ 分别是用户 `u` 和物品 `i` 的隐特征向量。  
- `λ` 是正则化系数，用于控制过拟合。

In [320]:
import numpy as np
from tqdm import tqdm

def matrix_factorization_by_bias(matrix, k, learning_rate=0.01, reg_param=0.1, iterations=1000, seed=42):
    """
    matrix: 评分矩阵
    k: 隐特征维度
    learning_rate: 学习率
    reg_param: 正则化参数
    iterations: 迭代次数
    seed: 随机数种子
    """
    np.random.seed(seed)
    rows, cols = matrix.shape
    
    # 初始化全局偏差
    u = np.mean(matrix[matrix > 0])  # 所有评分的全局平均分
    
    # 物品和用户的偏差初始化
    b_i = np.mean(matrix, axis=0)  # 物品偏差系数
    b_u = np.mean(matrix, axis=1)  # 用户偏差系数
    
    # 随机初始化隐因子矩阵
    user_matrix = np.random.rand(rows, k)
    item_matrix = np.random.rand(k, cols)

    # 迭代训练
    pbar = tqdm(range(iterations), desc="Training", ncols=100, unit='iter')
    
    for iter in pbar:
        total_loss = 0

        # 遍历每个评分
        for i in range(rows):
            for j in range(cols):
                if matrix[i, j] > 0:  # 只对非零评分进行优化
                    Prediction = u + b_i[j] + b_u[i] + np.dot(user_matrix[i, :], item_matrix[:, j])
                    error = matrix[i, j] - Prediction

                    # 计算梯度
                    user_grad = -2 * error * item_matrix[:, j] + 2 * reg_param * user_matrix[i, :]
                    item_grad = -2 * error * user_matrix[i, :] + 2 * reg_param * item_matrix[:, j]

                    # 更新隐因子矩阵
                    user_matrix[i, :] -= learning_rate * user_grad
                    item_matrix[:, j] -= learning_rate * item_grad

                    # 更新物品和用户的偏差
                    b_i[j] -= learning_rate * (-2 * error + 2 * reg_param * b_i[j])
                    b_u[i] -= learning_rate * (-2 * error + 2 * reg_param * b_u[i])

                    # 累加误差
                    total_loss += error ** 2

        # 更新进度条
        pbar.set_postfix({"Loss": total_loss})

    return u, b_i, b_u, user_matrix, item_matrix


In [321]:
#构建矩阵
matrix = generate_random_matrix(rows = 8,cols = 8,min_value = 0, max_value = 5,seed = 42)
matrix

array([[3, 4, 2, 4, 4, 1, 2, 2],
       [2, 4, 3, 2, 5, 4, 1, 3],
       [5, 5, 1, 3, 4, 0, 3, 1],
       [5, 4, 3, 0, 0, 2, 2, 1],
       [3, 3, 5, 5, 5, 2, 3, 3],
       [0, 2, 4, 2, 4, 0, 1, 3],
       [0, 3, 5, 1, 1, 0, 1, 4],
       [1, 3, 3, 3, 3, 4, 2, 5]])

In [333]:
#进行预测
u, b_i, b_u, user_matrix, item_matrix = matrix_factorization_by_bias(matrix,5, learning_rate = 0.01, reg_param=0.05,iterations = 1000, seed = 42)

Training: 100%|██████████████████████████████████| 1000/1000 [00:01<00:00, 614.01iter/s, Loss=0.767]


In [334]:
#预测矩阵
pd.DataFrame(u + b_i + b_u + np.dot(user_matrix, item_matrix))

,0,1,2,3,4,5,6,7
0,3.004081,4.124125,2.311388,3.980789,4.756490,1.008556,2.200382,2.240268
1,1.846504,3.834901,2.959681,1.929690,5.333630,3.449929,0.938230,3.063651
2,4.712637,4.913300,1.147050,2.951386,4.533641,1.516916,2.650857,1.165920
3,4.684745,4.178587,3.115767,2.409819,4.150500,1.685744,2.029596,1.257637
4,2.287087,2.545169,4.248641,4.177732,4.921215,1.096310,2.170004,2.593234
5,1.540449,2.618888,4.368397,2.394953,4.834507,3.171546,1.278511,3.438597
6,2.501700,3.146114,5.026820,1.183469,1.912712,3.712821,1.069909,4.208171
7,0.802794,3.047306,3.078525,2.771196,3.555536,3.461731,1.762797,4.808541


In [335]:
# 初始矩阵 original matrix
pd.DataFrame(matrix)

,0,1,2,3,4,5,6,7
0,3,4,2,4,4,1,2,2
1,2,4,3,2,5,4,1,3
2,5,5,1,3,4,0,3,1
3,5,4,3,0,0,2,2,1
4,3,3,5,5,5,2,3,3
5,0,2,4,2,4,0,1,3
6,0,3,5,1,1,0,1,4
7,1,3,3,3,3,4,2,5


### 4 小结
通过引入这些偏差项和隐特征，推荐系统能够更好地捕捉用户和物品之间的个性化差异，从而提高推荐结果的精度和多样性。这种方法帮助消除了由用户和物品的评分差异带来的偏差，使得推荐系统能够更精准地反映用户的实际偏好。